<a href="https://colab.research.google.com/github/ersilia-os/auto-tabnet/blob/main/TabNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install optuna
!pip install umap
!pip install umap-learn
!pip install pytorch-tabnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 KB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 8.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for umap: filename=umap-0.1.1-py3-none-any.whl size=3565 sha256=a2cca22940aa4db53c9f1cb49ee7e1f1d96d56c486c3cf3015b64b8cea214ecf
  Stored in directory: /root/.cache/pip/wheels/d4/13/91/2e752dc8dab5df027854bd33d2b65e1dc5cdc107fd1133990f
Successfully built umap
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 KB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from pytorch_tabnet.pretraining import TabNetPretrainer
from pytorch_tabnet.tab_model import TabNetClassifier
from pytorch_tabnet.metrics import Metric
import torch
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import QuantileTransformer
from sklearn import preprocessing
import torch.optim as optim
import torch.nn.functional as F
from sklearn import datasets
from sklearn import model_selection
from sklearn.decomposition import PCA
import os
from pathlib import Path
from torch.optim.lr_scheduler import ReduceLROnPlateau
import random
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
datax=pd.read_csv('/content/drive/MyDrive/Gesture_Data.csv')
li=['feature_'+str(i) for i in range(28)]
li.append("output")
datax.columns=li
datax.dropna(how='any',inplace=True)
for col in datax.columns:
  datax[col].astype('float32')
num = [*(x for x in datax.columns if x.isnumeric())]
for col in num:
    sel = QuantileTransformer(n_quantiles=1000,random_state=0,output_distribution='normal')
    sel.fit(datax[col].to_numpy().reshape(-1,1))
    datax[col] = sel.transform(datax[col].to_numpy().reshape(-1,1))

In [5]:
datax.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,output
0,0.07817,0.76526,0.07493,0.36647,0.69386,0.35531,0.71839,0.46066,40,14,...,89,0.00685,0.02819,0.00718,0.01859,0.76616,1.20575,0.76821,1.12635,1
1,0.06535,0.78984,0.07117,0.38040,0.74147,0.35050,0.73124,0.45126,44,14,...,88,0.00683,0.02827,0.00668,0.01906,0.74031,1.26830,0.74396,1.13324,1
2,0.06517,0.80187,0.06258,0.39622,0.74955,0.34247,0.76792,0.44406,46,15,...,86,0.00684,0.02981,0.00721,0.01841,0.75492,1.29168,0.74225,1.11862,1
3,0.06456,0.87456,0.07095,0.37056,0.71750,0.34110,0.74416,0.44714,47,15,...,82,0.00746,0.02774,0.00695,0.01738,0.73082,1.34581,0.77579,1.09954,1
4,0.07624,0.76268,0.06660,0.33706,0.69465,0.37617,0.75712,0.48599,42,17,...,85,0.00756,0.02682,0.00747,0.01779,0.74330,1.32099,0.78662,1.14457,1


In [6]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(datax, test_size=0.1)

In [10]:
X_train = train.iloc[:,:-1].to_numpy()
y_train = train['output'].squeeze()
X_test  = test.iloc[:,:-1].to_numpy()

In [11]:
tabnet_params = dict(
    n_d=64, n_a=64, n_steps=5,
    gamma=1.5, n_independent=2, n_shared=2,    
    cat_emb_dim=1,
    lambda_sparse=1e-4, momentum=0.3, clip_value=2.,
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    scheduler_params = {"gamma": 0.95,
                     "step_size": 20},
    scheduler_fn=torch.optim.lr_scheduler.StepLR, epsilon=1e-15
)

In [12]:
clf = TabNetClassifier(**tabnet_params)

/usr/local/lib/python3.8/dist-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


In [19]:
classifier = TabNetClassifier(verbose=0,seed=42)
classifier.fit(X_train=X_train, y_train=y_train,
               patience=5,max_epochs=100,
               eval_metric=['auc'])

predictions = classifier.predict_proba(X_test)[:,1]    


/usr/local/lib/python3.8/dist-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


In [ ]:
# save tabnet model
saving_path_name = "tabnet_model_test_"
saved_filepath = clf.save_model(saving_path_name)         

In [ ]:
loaded_clf = TabNetClassifier()
loaded_clf.load_model('/content/tabnet_model_test_9.zip')
y_pred=loaded_clf.predict_proba(X_test)
test_acc = roc_auc_score(y_score=y_pred[:,1],y_true=y_test)
print(test_acc)

In [16]:
predictions

array([3.00862819e-01, 2.45288197e-06, 2.27237232e-02, 9.92795885e-01,
       2.28544162e-03, 2.26500951e-08, 9.52023023e-04, 1.93406956e-03,
       9.99423623e-01, 9.98595297e-01, 2.67691398e-08, 9.99217391e-01,
       9.30204690e-01, 1.38154301e-05, 2.99012542e-01, 8.93136799e-01,
       8.33437871e-03, 3.75248902e-02, 9.98939097e-01, 9.97732520e-01,
       3.64331063e-04, 9.99989033e-01, 8.82415593e-01, 3.49201895e-02,
       9.47538356e-05, 9.97696221e-01, 7.95180261e-01, 9.82551694e-01,
       9.26288267e-05, 1.28818704e-02, 9.99757707e-01, 7.55235553e-02,
       6.03912849e-05, 9.98850822e-01, 4.81041097e-05, 9.84081745e-01,
       9.99644876e-01, 7.01681733e-01, 4.80172013e-10, 6.99209886e-06,
       9.92003322e-01, 9.99324799e-01, 5.11507802e-02, 2.44442163e-06,
       1.62560511e-02, 5.77241553e-06, 9.99079227e-01, 9.88328040e-01,
       1.11415800e-04, 9.98541951e-01, 4.80392446e-05, 9.99959588e-01,
       9.85479474e-01, 2.51540740e-04, 2.08368134e-02, 2.31770813e-04,
      

In [14]:
sample.iloc[:,1:] = predictions
sample.to_csv('submission.csv',index=False)

NameError: ignored